In [ ]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append("./scripts/") # put the path where the scripts are stored

!pip install torch==1.1.0
!pip install pytorch-pretrained-bert==0.6.2

!python -m spacy download en_core_web_sm
!python -m spacy download en

In [ ]:
'''
creating the dataset
'''
from convertBrat import *

kag_addr = "./"
folder = kag_addr+"./data/brat"
outdirr = "/output/"+"./data/tsv"

os.makedirs("%s/%s" % (outdirr, "train"))
os.makedirs("%s/%s" % (outdirr, "dev"))
os.makedirs("%s/%s" % (outdirr, "test"))

train, dev, test=read_splits(kag_addr+"./data/train.ids", kag_addr+"./data/dev.ids", kag_addr+"./data/test.ids")

onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]

for filename in onlyfiles:
    if filename.endswith(".txt"):
        base=re.sub(".txt", "", filename.split("/")[-1])
        print(base)

        textFile="%s/%s.txt" % (folder, base)
        annFile="%s/%s.ann" % (folder, base)

        outDir=None
        if base in train:
            outDir=os.path.join(outdirr, "train")
        elif base in dev:
            outDir=os.path.join(outdirr, "dev")
        elif base in test:
            outDir=os.path.join(outdirr, "test")

        outFile=os.path.join(outDir, "%s.tsv" % base)

        anns=read_ann(annFile)
        read_txt(textFile, anns, outFile)

In [ ]:
'''
Make bert_feature_vectors from the dataset
'''
from return_bert_features import *

train_list = list(train.keys())
dev_list = list(dev.keys())
test_list = list(test.keys())

def loop(i,dataset):
    args = {}
    args["file"]="/output"+"/data/tsv/"+dataset+"/"+i
    args["model_path"]="bert-base-cased"
    args["output"]="/output"+"/data/bert/"+dataset+"/"+i

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('')
    print("********************************************")
    print("Running on: {}".format(device))
    print("Using the following model: {}".format(args["model_path"]))
    print("********************************************")
    print('')

    sents, orig_lines = read_sentences([args["file"]])

    berts = get_bert_representations(args["model_path"], sents)

    with open(args["output"], "w", encoding="utf-8") as out:
        for idx, bert in enumerate(berts):
            orig=orig_lines[idx]
            for i in range(len(orig)):
                out.write("%s\t%s\n" % (orig[i], ' '.join(str(x) for x in bert[i])))
            out.write("\n")


dataset="train"
os.makedirs("/output/data/bert/"+dataset+"/")
for i in train_list:
    i = i+".tsv"
    loop(i,dataset)

dataset="dev"
os.makedirs("/output/data/bert/"+dataset+"/")
for i in dev_list:
    i = i+".tsv"
    loop(i,dataset)

dataset="test"
os.makedirs("/output/data/bert/"+dataset+"/")
for i in test_list:
    i = i+".tsv"
    loop(i,dataset)

In [ ]:
'''
Get the gutebnerg vectors for using the word embeddings
'''
%%bash
wget http://people.ischool.berkeley.edu/~dbamman/data/guten.vectors.txt.gz
gunzip guten.vectors.txt.gz
# mkdir ./data
mv guten.vectors.txt ./data/